In [3]:
import dask.dataframe as dd
import numpy

In [16]:
df = dd.read_csv('cosine.csv')

,1.0,0.11958186451359087,0.050788403766058976,0.08308729744733989,0.2900498381909672,0.3283190490311336,0.3236942787221089,0.20519274920879238,0.08375032197501475,0.3001891414685356,...,0.13107951714881128,0.25407353409196,0.17035430547833053,0.1527366831606597,0.13338472575636762,0.22866490251057103,0.1311486823722008,0.13783448216916472,0.3053756651345927,Unnamed: 943
0,0.119582,1.000000,0.104092,0.164484,0.072657,0.175854,0.096470,0.075242,0.087820,0.131937,...,0.149250,0.198495,0.394685,0.259404,0.202697,0.191010,0.138802,0.170524,0.087974,NaN
1,0.050788,0.104092,1.000000,0.307379,0.026295,0.062776,0.077204,0.039315,0.077869,0.050313,...,0.052537,0.147268,0.034371,0.138783,0.000000,0.155616,0.049118,0.153153,0.032688,NaN
2,0.083087,0.164484,0.307379,1.000000,0.040403,0.060083,0.062088,0.101174,0.132611,0.077921,...,0.046393,0.120529,0.196116,0.140597,0.039379,0.167981,0.130120,0.215374,0.074224,NaN
3,0.290050,0.072657,0.026295,0.040403,1.000000,0.195667,0.297408,0.204108,0.035924,0.183979,...,0.096143,0.095891,0.087433,0.126871,0.045124,0.229951,0.103104,0.153093,0.258780,NaN
4,0.328319,0.175854,0.062776,0.060083,0.195667,1.000000,0.401021,0.178790,0.174847,0.453544,...,0.092333,0.139118,0.220937,0.126834,0.109428,0.254636,0.141702,0.197637,0.248531,NaN
